# Mount Drive

In [1]:
from google.colab import drive

drive.mount('/content/drive/')

Mounted at /content/drive/


# Import Libraries

In [ ]:
!pip install git+git://github.com/rasbt/mlxtend.git

import pandas as pd
import numpy as np
from mlxtend.preprocessing import TransactionEncoder
from mlxtend.frequent_patterns import association_rules
from mlxtend.frequent_patterns import fpgrowth



  Cloning git://github.com/rasbt/mlxtend.git to /tmp/pip-req-build-echi5msn
  Running command git clone -q git://github.com/rasbt/mlxtend.git /tmp/pip-req-build-echi5msn
  Created wheel for mlxtend: filename=mlxtend-0.18.0-py2.py3-none-any.whl size=1588709 sha256=f11f79defbfe828a67b643ef736fee2bc7cbebafae2186c032cb98d6184204db
  Stored in directory: /tmp/pip-ephem-wheel-cache-if8gsepk/wheels/85/e4/4c/ee71547ac9ea223b07fe8f55b0e5f71536a6a34ae3480205f3
Successfully built mlxtend


# Initialize dataframe

In [ ]:
data = pd.read_csv('/content/drive/My Drive/datasets/dataset_transaksipenjualan.csv')
data.head()

,Tanggal SPS,Tanggal Bayar,No. SPS,Saluran,ID Pembeli,KAB/ Kota,Nama Komoditi,Jenis Komoditi,Merk Komoditi,Kemasan,Satuan,Kolli Terjual,Kuantum Terjual (Kg/L),Harga per Kemasan,Nominal Penjualan
0,01-02-2019,01-02-2019,02130/01/2019/09001/SPST,KOPERASI/ASOSIASI,77844,KOTA ADM. JAKARTA SELATAN,DAGING KERBAU,FOREQUARTER,DAGING KITA,18,Kg,1,18,1098000,1098000
1,01-02-2019,01-02-2019,02010/01/2019/09001/SPST,TOKO/AGEN/PENGECER,72580,KOTA ADM. JAKARTA TIMUR,DAGING KERBAU,FOREQUARTER,DAGING KITA,20,Kg,5,100,1440000,7200000
2,17-12-2018,01-02-2019,01200/12/2018/09001/SPST,HOREKA,55107,KAB. TANGERANG,DAGING KERBAU,FOREQUARTER,DAGING KITA,20,Kg,12,240,1380000,16560000
3,01-02-2019,01-02-2019,02007/01/2019/09001/SPST,HOREKA,55107,KAB. TANGERANG,DAGING KERBAU,FOREQUARTER,DAGING KITA,20,Kg,7,140,1380000,9660000
4,01-03-2019,01-02-2019,00007/01/2019/09001/SPST,RPK-NON BPNT/Program Sembako,67725,KOTA ADM. JAKARTA PUSAT,DAGING KERBAU,HINDQUARTER,DAGING KITA,1,Kg,75,75,70000,5250000


In [ ]:
# Select Column
data = data.sort_values(by=['Tanggal Bayar', 'No. SPS'])[['No. SPS','Nama Komoditi', 'Jenis Komoditi', 'Merk Komoditi']]
data.head()

data["Komoditi"] = data["Nama Komoditi"] + " - " + data["Jenis Komoditi"] + " (" + data["Merk Komoditi"] + ")"
data = data.drop(['Nama Komoditi', 'Jenis Komoditi', 'Merk Komoditi'], axis=1)
data.head()



,No. SPS,Komoditi
13,00001/01/2019/09001/SPST,DAGING KERBAU - HINDQUARTER (DAGING KITA)
14,00001/01/2019/09001/SPST,MINYAK GORENG - MINYAK GORENG KITA (MINYAK GOR...
15,00001/01/2019/09020/SPST,BERAS - THAILAND 5% (BERAS POLOS)
12,00002/01/2019/09001/SPST,DAGING KERBAU - HINDQUARTER (DAGING KITA)
11,00003/01/2019/09001/SPST,DAGING KERBAU - HINDQUARTER (DAGING KITA)


# Change to standard format for FPGROWTH

In [ ]:
# Replace Komoditi values
komoditi_list = data['Komoditi'].unique()
komoditi_list

replace_list = []
for i in range(len(komoditi_list)):
  replace_list.append('I'+str(i + 1))
replace_list

# Create legend table
defdata = {
    'Kode': replace_list,
    'Komoditi': komoditi_list,
}

# Replace
data = data.replace(komoditi_list, replace_list)
data

# Replace SPS values as Transaction
sps_list = data['No. SPS'].unique()
sps_list

replace_sps_list = []
for i in range(len(sps_list)):
  replace_sps_list.append('T'+str(i + 1).zfill(4))
replace_sps_list

data = data.replace(sps_list, replace_sps_list)
data

# Rename columns
data = data.rename(columns={'No. SPS': 'TID', 'Komoditi': 'Item'})
data

,TID,Item
13,T0001,I1
14,T0001,I2
15,T0002,I3
12,T0003,I1
11,T0004,I1
...,...,...
9800,T5704,I7
9798,T5705,I7
9799,T5706,I7
9797,T5707,I7


In [ ]:
# Group
data_preprocess = data.groupby('TID')['Item'].apply(','.join).reset_index()
data_preprocess

data_preprocess.sort_values('TID')


,TID,Item
0,T0001,"I1,I2"
1,T0002,I3
2,T0003,I1
3,T0004,I1
4,T0005,"I4,I2"
...,...,...
5703,T5704,I7
5704,T5705,I7
5705,T5706,I7
5706,T5707,I7


# Implement Algorithm

In [ ]:

te = TransactionEncoder()
data_preprocess['Item'].tolist()
print(data_preprocess['Item'].tolist())
print(len(data_preprocess['Item'].tolist()))

# Change to suitable format for the algorithm
new_list = []
for i in data_preprocess['Item'].tolist():
  if len(i.split(',')) > 2:
    new_list.append(i.split(','))
print(new_list)
print(len(new_list))

te_ary = te.fit(new_list).transform(new_list)
df = pd.DataFrame(te_ary, columns=te.columns_)
df


['I1,I2', 'I3', 'I1', 'I1', 'I4,I2', 'I5,I6,I2', 'I5', 'I1', 'I7', 'I7', 'I7', 'I7', 'I1,I8', 'I1', 'I1', 'I6', 'I9', 'I5,I4,I5', 'I5', 'I10,I11', 'I11', 'I11', 'I1', 'I7', 'I7', 'I9', 'I6', 'I5,I5', 'I1', 'I1,I12', 'I7', 'I7', 'I7,I7', 'I1', 'I1', 'I7', 'I7', 'I9', 'I7', 'I7', 'I7', 'I7', 'I1', 'I7', 'I13,I5,I6,I2,I8', 'I1,I6', 'I4,I5', 'I1', 'I1', 'I1', 'I5,I4,I1,I11,I5,I6,I2', 'I1', 'I1', 'I4', 'I1,I6,I2', 'I5,I6,I2', 'I4,I5', 'I14,I15,I1', 'I16', 'I7', 'I1', 'I1', 'I1', 'I7', 'I17', 'I7', 'I5,I6,I2,I8', 'I17', 'I17', 'I5,I4,I5', 'I1,I2,I8', 'I1', 'I4', 'I1', 'I11', 'I6', 'I1', 'I5,I6,I2,I8', 'I6,I8', 'I6', 'I1,I2', 'I4,I5', 'I1', 'I4,I15,I18,I11,I5,I6,I2,I8', 'I1', 'I17', 'I19', 'I11', 'I1', 'I9', 'I9', 'I1', 'I1', 'I1', 'I7', 'I7', 'I20,I7', 'I7', 'I20', 'I12,I11,I5,I6,I2,I8', 'I1,I6', 'I1', 'I1,I6', 'I4,I6,I2', 'I1', 'I1', 'I4,I1,I6', 'I13,I6,I8', 'I5,I6,I2,I8', 'I6', 'I5,I6,I2', 'I15,I1,I21,I2,I8', 'I16', 'I1', 'I1', 'I1', 'I1', 'I5,I4,I15,I1,I5,I6,I2', 'I5,I4,I5', 'I5,I15,I6', 

,I1,I10,I11,I12,I13,I14,I15,I17,I18,I19,I2,I20,I21,I22,I23,I24,I25,I26,I27,I28,I29,I30,I31,I32,I33,I34,I35,I36,I37,I38,I39,I4,I42,I44,I5,I6,I7,I8
0,False,False,False,False,False,False,False,False,False,False,True,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,True,True,False,False
1,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,True,False,False,True,False,False,False
2,False,False,False,False,True,False,False,False,False,False,True,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,True,True,False,True
3,True,False,True,False,False,False,False,False,False,False,True,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,True,False,False,True,True,False,False
4,True,False,False,False,False,False,False,False,False,False,True,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,True,False,False
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1053,False,False,True,False,False,False,False,True,False,False,True,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False
1054,False,False,True,False,False,False,False,True,False,False,True,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,True,False,False,False,False,False,False
1055,False,False,True,False,False,False,False,True,False,False,True,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,True,False,False,False,False,False,False
1056,False,False,False,False,False,False,False,True,False,False,True,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,True,False,False,True


In [ ]:
f_itemsets = fpgrowth(df, min_support=0.3, use_colnames=True)
f_itemsets

,support,itemsets
0,0.721172,(I2)
1,0.525520,(I6)
2,0.368620,(I5)
3,0.433837,(I8)
4,0.439509,(I11)
5,0.403592,"(I6, I2)"
6,0.346881,"(I8, I2)"
7,0.315690,"(I11, I2)"


# Create Association Rule

In [ ]:
association_rules(f_itemsets, metric="confidence", min_threshold=0.7)

,antecedents,consequents,antecedent support,consequent support,support,confidence,lift,leverage,conviction
0,(I6),(I2),0.525520,0.721172,0.403592,0.767986,1.064913,0.024601,1.201770
1,(I8),(I2),0.433837,0.721172,0.346881,0.799564,1.108701,0.034009,1.391109
2,(I11),(I2),0.439509,0.721172,0.315690,0.718280,0.995989,-0.001271,0.989733


In [ ]:
pd.DataFrame(defdata, columns=['Kode', 'Komoditi'])

,Kode,Komoditi
0,I1,DAGING KERBAU - HINDQUARTER (DAGING KITA)
1,I2,MINYAK GORENG - MINYAK GORENG KITA (MINYAK GOR...
2,I3,BERAS - THAILAND 5% (BERAS POLOS)
3,I4,BERAS - KHUSUS (BERAS PANDAN WANGI 5KG)
4,I5,BERAS - PREMIUM 10% (BERAS KITA PREMIUM)
5,I6,GULA - GKP EKS RAWSUGAR PT.JMR (MANIS KITA)
6,I7,DAGING KERBAU - FOREQUARTER (DAGING KITA)
7,I8,TEPUNG TERIGU - TERIGU KITA (TERIGU KITA)
8,I9,BERAS - VIETNAM 5% (BERAS POLOS)
9,I10,BERAS - KHUSUS (BERASKITA BERAS HITAM)
